## Masked Autoencoders（MAE）的可视化

本次教程改编自Masked Autoencoders (MAE)的[官方GitHub代码](https://github.com/facebookresearch/mae)。

MAE模型向一张输入图像中加入随机掩码，随后使用一个视觉Transformer模型恢复掩码部分的图像，以此进行自监督学习。

通过这种方式，MAE仅需要无标签的图像就可以进行训练，可作为下游任务的预训练模型。

<img src="images/mae.png" width="50%" height="50%">


### 准备工作

#### 1. 安装timm库

timm全称为Py**T**orch **Im**age **M**odels，包含了各种计算机视觉任务的模型和方法。

本次教程将使用timm库中Vision Transformer模型提供的PatchEmbed和Block方法。

其中，PatchEmbed将图像进行分块，并将每一块映射为一个特征向量。Block则为Transformer层的实现。



In [ ]:
!pip install timm==0.4.5

#### 2. 下载预训练模型
视觉Transformer模型较大，预训练的模型可以通过[清华云盘](https://cloud.tsinghua.edu.cn/f/f551ef07d2ce42e39001/?dl=1)下载。

请将下载后的模型文件(mae_visualize_vit_large.pth)放到pretrained_model目录下。

#### 3. 导入所需要的库

In [ ]:
import torch
import numpy as np

import matplotlib.pyplot as plt
import cv2

import models_mae

### 定义所需函数

In [ ]:
# ImageNet数据集图像像素的均值和标准差
# MAE读取图像时会使用该均值和标准差进行归一化
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def load_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.

    assert img.shape == (224, 224, 3)

    # 归一化
    img = img - imagenet_mean
    img = img / imagenet_std
    return img

def show_image(image, title=''):
    assert image.shape[2] == 3  # 输入为RGB图像
    plt.imshow(torch.clip((image * imagenet_std + imagenet_mean) * 255, 0, 255).int())
    plt.title(title, fontsize=16)
    plt.axis('off')
    return

def prepare_model(chkpt_dir, arch='mae_vit_large_patch16'):
    # Python中的getattr(object, name)函数用于从一个对象中获取命名为name的属性
    # getattr(models_mae, arch)返回的是函数本身，之后再加括号()得到该函数返回值
    model = getattr(models_mae, arch)()
    checkpoint = torch.load(chkpt_dir, map_location='cpu')
    msg = model.load_state_dict(checkpoint['model'], strict=False)
    print(msg)
    return model

def run_one_image(img, model):
    # x: hwc 高-宽-通道
    x = torch.tensor(img)
    # 添加batch维度
    x = x.unsqueeze(dim=0)

    # 爱因斯坦和，使用符号标记对张量维度进行变换
    # 等价于x.permute(0, 3, 1, 2)
    x = torch.einsum('nhwc->nchw', x)

    # 运行MAE
    loss, y, mask = model(x.float(), mask_ratio=0.75)  # 对图像中75%的部分添加掩码
    y = model.unpatchify(y)  # 对分块后的图像进行复原
    y = torch.einsum('nchw->nhwc', y).detach().cpu()

    # visualize the mask
    mask = mask.detach()
    mask = mask.unsqueeze(-1).repeat(1, 1, model.patch_embed.patch_size[0]**2 *3)  # (N, H*W, p*p*3)
    mask = model.unpatchify(mask)
    mask = torch.einsum('nchw->nhwc', mask).detach().cpu()
    
    x = torch.einsum('nchw->nhwc', x)

    # 加入掩码的图像
    im_masked = x * (1 - mask)  # mask中1表示此处加入掩码

    # 重建后的图像与原图叠加（原图中未加入掩码的部分保留）
    im_paste = x * (1 - mask) + y * mask

    # 可视化
    plt.rcParams['figure.figsize'] = [24, 24]

    plt.subplot(1, 4, 1)
    show_image(x[0], "original")

    plt.subplot(1, 4, 2)
    show_image(im_masked[0], "masked")

    plt.subplot(1, 4, 3)
    show_image(y[0], "reconstruction")

    plt.subplot(1, 4, 4)
    show_image(im_paste[0], "reconstruction + visible")

    plt.show()

### 载入预训练MAE模型

In [ ]:
chkpt_path = 'pretrained_model/mae_visualize_vit_large.pth'
model_mae = prepare_model(chkpt_path)
print('Model loaded.')

### 读取一张图像

In [ ]:
img = load_image('images/fox.jpg')
plt.rcParams['figure.figsize'] = [5, 5]
show_image(torch.tensor(img))

### Run MAE on the image

In [ ]:
torch.manual_seed(2)  # 加入随机种子固定掩码的位置
print('MAE with pixel reconstruction:')
run_one_image(img, model_mae)

### 试试另一张图像

In [ ]:
torch.manual_seed(2)
img = load_image('images/airplane.png')
run_one_image(img, model_mae)

### 用不同风格的图像验证模型的泛化能力

In [ ]:
torch.manual_seed(6)
img = load_image('images/totoro.png')
run_one_image(img, model_mae)